# Initialization

In [14]:
%run 050-init.ipynb

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 17.0.8
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.
--2023-08-30 15:03:07--  https://raw.githubusercontent.com/meddash-cloud/meddash-public-datasets/main/data/walmart_stock.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90266 (88K) [text/plain]
Saving to: ‘walmart_stock.csv.1’

walmart_stock.csv.1 100%[===================>]  88.15K  --.-KB/s    in 0.02s   

2023-08-30 15:03:07 (4.27 MB/s) - ‘wal

# get csv file from remote

In [ ]:
# Split CSV into chunks

In [ ]:
%%bash
rm -rf csv_chunks
mkdir -p csv_chunks
split -d -l 40 walmart_stock.csv csv_chunks/chunk_
for f in `ls csv_chunks`; do
mv csv_chunks/$f csv_chunks/$f.csv
done

In [15]:
BUCKET_NAME="spark-stream-on-minio-csvs"
minio_url = "minio-service.kubeflow.svc.cluster.local:9000"
minio_key = "minio"
minio_secret = "minio123"
minio_config = {
        "endpoint": minio_url,
        "access_key": "minio",
        "secret_key": "minio123",
        "secure": False,
        }

In [17]:
import time
from threading import Thread
from minio import Minio
from minio.error import S3Error
import os
import glob

class UploadCSVChunksToMinio(Thread):
    def __init__(self, csv_dir_path=None):
        super(UploadCSVChunksToMinio, self).__init__()

        self.csv_files = sorted(glob.glob("{}/*.csv".format("csv_chunks")))
        print(self.csv_files)
        print ("connecting to minio {}".format(minio_url))
        self.minio_client = Minio(**minio_config)
        
        print("try to find bucket {}".format(BUCKET_NAME))
        found = self.minio_client.bucket_exists(BUCKET_NAME)
        print("found", found)
        if not found:
            self.minio_client.make_bucket(BUCKET_NAME)
        else:
            print("Bucket '{}' already exists".format(BUCKET_NAME))


    def upload_file_to_minio(self, upload_file_path):
        file_name = os.path.basename(upload_file_path)
        self.minio_client.fput_object(BUCKET_NAME, file_name, upload_file_path)
    def  deleteAllCSVs(self):
        # Delete using "remove_object"
        objects_to_delete = self.minio_client.list_objects(BUCKET_NAME, prefix="/", recursive=True)
        for obj in objects_to_delete:
            self.minio_client.remove_object(BUCKET_NAME, obj.object_name)
    def run(self):
        i=0
        self.deleteAllCSVs()
        for csv_file in self.csv_files:
            self.upload_file_to_minio(csv_file)
            time.sleep(10)
UploadCSVChunksToMinio(csv_dir_path="csv_chunks").start()

['csv_chunks/chunk_00.csv', 'csv_chunks/chunk_01.csv', 'csv_chunks/chunk_02.csv', 'csv_chunks/chunk_03.csv', 'csv_chunks/chunk_04.csv', 'csv_chunks/chunk_05.csv', 'csv_chunks/chunk_06.csv', 'csv_chunks/chunk_07.csv', 'csv_chunks/chunk_08.csv', 'csv_chunks/chunk_09.csv', 'csv_chunks/chunk_10.csv', 'csv_chunks/chunk_11.csv', 'csv_chunks/chunk_12.csv', 'csv_chunks/chunk_13.csv', 'csv_chunks/chunk_14.csv', 'csv_chunks/chunk_15.csv', 'csv_chunks/chunk_16.csv', 'csv_chunks/chunk_17.csv', 'csv_chunks/chunk_18.csv', 'csv_chunks/chunk_19.csv', 'csv_chunks/chunk_20.csv', 'csv_chunks/chunk_21.csv', 'csv_chunks/chunk_22.csv', 'csv_chunks/chunk_23.csv', 'csv_chunks/chunk_24.csv', 'csv_chunks/chunk_25.csv', 'csv_chunks/chunk_26.csv', 'csv_chunks/chunk_27.csv', 'csv_chunks/chunk_28.csv', 'csv_chunks/chunk_29.csv', 'csv_chunks/chunk_30.csv', 'csv_chunks/chunk_31.csv']
connecting to minio minio-service.kubeflow.svc.cluster.local:9000
try to find bucket spark-stream-on-minio-csvs
found True
Bucket 'spar

In [ ]:
%%bash
rm -rf output checkpoint

In [ ]:
import os

cwd_ = os.getcwd()
minio_dir_path = "s3a://{}/*".format(BUCKET_NAME)
print(minio_dir_path)

spark.sql("set spark.sql.streaming.schemaInference=true")

data = spark\
.readStream\
.format("csv")\
.load(minio_dir_path)\
.writeStream\
.format("csv")\
.option("checkpointLocation", "checkpoint/")\
.start("output/")\
.awaitTermination()

s3a://spark-stream-on-minio-csvs/*
